In [ ]:
import pandas as pd
from common import *

# Compare iSnobal SWI to USGS discharge gauge at Almont 

## USGS 

Source: https://nwis.waterdata.usgs.gov/co/nwis/dv/?site_no=09112500&agency_cd=USGS&amp;referred_module=sw

In [ ]:
PD_OPTIONS = dict(
    comment='#',
    parse_dates=True,
    index_col=0,
    names=['Date', 'Discharge'],
    header=0,
    sep='\t',
    dtype={'Discharge': np.float64},
    usecols=[2,3],
)
data = pd.read_csv(
    DATA_DIR / 'USGS-SWI-Almont.csv', 
    **PD_OPTIONS
)

data.Discharge = data.Discharge * 60 * 60 * 24

## SNOBAL

In [ ]:
client = start_cluster(5, 16)
client_ip_and_port(client)

In [ ]:
# Convert SNOBAL grid cell from meter to square feet
CELL_AREA = (50.0 * 3.28084)**2
# MM in one foot
FT_IN_MM = 304.8
# Convert to area
CELL_AREA_IN_MM = CELL_AREA / FT_IN_MM

In [ ]:
snobal_em = xr.open_mfdataset(
    (SNOBAL_DIR / 'wy*/erw/*/em.nc').as_posix(),
    data_vars=['SWI'],
    parallel=True,
)
snobal_em.coords['mask'] = (('y', 'x'), cbrfc_zones())
snobal_em['time'] = snobal_em.time + np.timedelta64(2,'h')

In [ ]:
swi = (
    snobal_em.where(snobal_em.mask != 0).SWI * CELL_AREA_IN_MM
).sum(['x', 'y']).compute()

In [ ]:
client.shutdown()

### Calculate added/removed daily SWE 

### SNOBAL 

### Moving 7-day average SNOBAL vs. USGS

In [ ]:
x_axis_fmt = mdates.DateFormatter('%d-%b-%y')

In [ ]:
plt.figure(dpi=300, figsize=(9,3.3))
ax = plt.gca()

swi.rolling(time=7, center=True).mean().plot(ax=ax, label='iSnobal Runoff', color='peru', alpha=0.7, lw=1)
data.rolling(7, center=True).mean().Discharge.plot(ax=ax, label='USGS Discharge', color='cadetblue', alpha=0.7, lw=1)

ax.xaxis.set_major_formatter(x_axis_fmt)
ax.set_xlim(left=data.index.min(), right=data.index.max())
ax.ticklabel_format(axis='y', useMathText=True)

plt.ylabel('7-Day Mean Output (mm)')
plt.legend();

In [ ]:
wy2018 = dict(time=slice("2017-10-01", "2018-09-30"))
wy2019 = dict(time=slice("2018-10-01", "2019-09-30"))
wy2020 = dict(time=slice("2019-10-01", "2020-09-30"))
wy2021 = dict(time=slice("2020-10-01", "2021-09-30"))